# swisslandstats preprocessing

We will create an extract of the Canton of Vaud (Switerland) derived from the [Swiss Land Statistics (SLS) datasets from the Swiss Federal Statistical Office](https://www.bfs.admin.ch/bfs/en/home/services/geostat/swiss-federal-statistics-geodata/land-use-cover-suitability/swiss-land-use-statistics.html). We will first use GNU Make to download the data:

In [ ]:
!snakemake -c1 -d .. -s ../Snakefile lulc_csv

Building DAG of jobs...
Nothing to be done (all requested files are present and up to date).
Complete log: .snakemake/log/2023-08-10T144454.450275.snakemake.log


This downloads the SLS inventory as a CSV file:

In [ ]:
!ls ../data/raw/lulc

AREA_NOAS04_17_181029.csv


Each row corresponds to one of the hectometric pixels that configure the Swiss territory, and features three main goups of columns [1], i.e.:

* the coordinates of the pixels centroid `E`, `N` in the LV95 coordinate reference system (CRS) or `X`, `Y` in the LV03 CRS
* columns starting with `FJ` denote the exact year when the observation was taken
* columns starting with `AS` denote the actual land use/land cover (LULC) code of each pixel

This is what the file looks like:

In [ ]:
!head ../data/raw/lulc/AREA_NOAS04_17_181029.csv

"E","N","X","Y","RELI","GMDE","FJ85","FJ97","FJ09","FJ18","AS85R_17","AS97R_17","AS09R_17","AS18_17","AS85R_4","AS97R_4","AS09R_4","AS18_4"
2485500,1109700,485500,109700,48551097,6611,1980,1992,2004,2012,10,10,10,10,3,3,3,3
2485500,1109800,485500,109800,48551098,6611,1980,1992,2004,2012,10,10,10,10,3,3,3,3
2485500,1109900,485500,109900,48551099,6611,1980,1992,2004,2012,10,10,10,10,3,3,3,3
2485500,1110000,485500,110000,48551100,6611,1980,1992,2004,2012,10,10,10,10,3,3,3,3
2485500,1110100,485500,110100,48551101,6611,1980,1992,2004,2012,10,10,10,10,3,3,3,3
2485600,1109800,485600,109800,48561098,6611,1980,1992,2004,2012,7,7,7,7,2,2,2,2
2485600,1109900,485600,109900,48561099,6611,1980,1992,2004,2012,10,10,10,10,3,3,3,3
2485600,1110000,485600,110000,48561100,6611,1980,1992,2004,2012,10,10,10,10,3,3,3,3
2485600,1110100,485600,110100,48561101,6611,1980,1992,2004,2012,10,10,10,10,3,3,3,3


We will now preprocess this data using the [swisslandstats-geopy](https://github.com/martibosch/swisslandstats-geopy) library:

In [ ]:
import swisslandstats as sls

Below are the parameters needed to run this notebook (to be filled directly in the notebook or using [papermill](https://github.com/nteract/papermill)):

In [ ]:
lulc_csv_filepath = "../data/raw/lulc/AREA_NOAS04_17_181029.csv"
lulc_column = "AS18_4"
dst_filepath = "../data/processed/veveyse-AS18_4.tif"
nominatim_query = "District de la Veveyse, Fribourg, Switzerland"

We will now:

1. read the data into a land data frame

In [ ]:
ldf = sls.read_csv(lulc_csv_filepath)

2. clip it to the extent of interest:

In [ ]:
extent_ldf = ldf.clip_by_nominatim(nominatim_query)

3. dump the LULC columns of interest into a geotiff file:

In [ ]:
extent_ldf.to_geotiff(dst_filepath, lulc_column)

## References

1. Bosch, M. (2019). swisslandstats-geopy: Python tools for the land statistics datasets from the Swiss Federal Statistical Office. Journal of Open Source Software, 4(41), 1511.